In [1]:
import os, sys, json
autocog_home = os.getenv("AUTOCOG_HOME", '/workspace/projects/autocog')
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.lm import OpenAI, Llama, TfLM
from autocog.architecture.utility import PromptTee

arch = CogArch(pipe=PromptTee(prefix='test', tee=sys.stdout))

llama_path = lambda x: "/workspace/models/llama/{}/ggml-model-{}.bin".format(*x)
model_kwargs = Llama.create(model_path=llama_path(('7B','q4_0')), n_ctx=2048)
arch.orchestrator.LMs.update({
  'text'     : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' :  0 }),
  'thought'  : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 10, 'temperature' : 1.0 }),
  'sentence' : Llama(**model_kwargs, completion_kwargs={ 'max_tokens' : 50, 'temperature' : 0.7 }),
}) # llama-cpp-python defaults: top_p=0.95, top_k=40, repeat_penalty=1.1

llama.cpp: loading model from /workspace/models/llama/7B/ggml-model-q4_0.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 2048
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  68.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 1026.00 MB per state)
llama_init_from_file: kv self size  = 1024.00 MB


In [2]:
cog = arch.load(tag='test', language='sta', program="""
entry(formulate):

formats:
- sentence: Properly formed sentence in english. When asked for a list of sentences, they should form a coherent paragraph.

prompt(formulate): You formulate an answer to a question. The comments are your own reflexion on the question.
- target(question) source(?question)
- target(comments) source(reflexion.comments)
> question(text): a user's question
> comments[{Nc}](thought): your reflexion based on a previous answer
> work[{Nw}](thought): work through the user's question, consider the comments if available.
> answer[{Na}](sentence): write down your answer
__next(reflexion): You will evaluate your answer and provide comments if the answer needs to be improved.

prompt(reflexion): You formulated an answer to the user question. Now, you will comment this answer then decide whether to formulate another answer or submit your current answer.
- target(question) source(?question)
- target(answer) source(formulate.answer)
> question(text): a user's question
> answer[{Na}](sentence): your current answer
> comments[{Nc}](thought): evaluate how this answer satisfy the question. Include pertinent comments that will help create a better answer if needed.
__next(formulate,submit): should you "formulate" a new answer or "submit" your current answer\

prompt(submit): 
- target(answer) source(reflexion.answer)
> answer[{Na}](sentence):  
__exit(answer): 

?Na=10
?Nw=20
?Nc=10
""")
# cog.model_dump()

In [3]:
await arch('test', question='What is existentialism?')



 === test[0] === 

You are a helpful AI assistant.  You formulate an answer to a question. The comments are your own reflexion on the question.
You are using an interactive questionnaire.
Follow this structure after the start prompt:
```
> question(text): a user's question
> comments[10](thought): your reflexion based on a previous answer
> work[20](thought): work through the user's question, consider the comments if available.
> answer[10](sentence): write down your answer
```
Each prompt expects one of the following formats:
- text: ASCII text in any form
- thought: your thoughts (a few words per lines)
- sentence: Sentences should use correct english in an academic style. When asked for a list of sentences, they should form a coherent paragraph.
Terminate each prompt with a newline. Use as many statement with `thought` format as needed.

start(record):
> question(text): What is existentialism?
> work[1](thought):  "Existentialism in philosophy, literature,
> work[2](thought):  art

{'answer': [' You can find a more complete definition on their website.',
  ' This is a philosophical position or movement, usually incompatible with metaphysics or religion that focuses on the issues raised by the lack of an absolute meaning to existence.',
  ' "Is man\'s life hereafter a reality? I don\'t know, but if it is we are very unlikely to find out in our lifetime so better enjoy the time you have now."',
  ' - Existentialism.org. It is important to understand that existentialism does not concern itself with whether a god exists or not, how the universe started etc., but rather questions the meaning of life and what humans are capable of doing given',
  ' the limitations that life brings with it. - Wikipedia, Existentialism',
  ' You can find a more complete definition on their website.',
  ' This is a philosophical position or movement, usually incompatible with metaphysics or religion that focuses on the issues raised by the lack of an absolute meaning to existence. "Is man